In [1]:
import os
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np 
from scipy.signal import welch
from detect_peaks import detect_peaks
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import entropy #seems non beneficial
from scipy.signal import hilbert

In [2]:
#Import Machine Learning Libraries
from pandas.plotting import scatter_matrix
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [3]:
def autocorr(x):
    #Subfunction of get_autocorr_values
    result = np.correlate(x, x, mode='full')
    return result[len(result)//2:]

def get_autocorr_values(UserInput):
    sig = UserInput['Signal Data of Interest']
    Tmax = UserInput['Time of Sampling']
    autocorr_values = autocorr(sig)
    x_values = np.array([Tmax * jj for jj in range(0, N)])
    x = {
        "X Values":x_values,
        "Autocorr Values": autocorr_values
        }
    return x

In [4]:
def GetSortedPeak(X,Y):
    #SubFunction for FrequencyDomainInformation
    max_peak_height = 0.1 * np.nanmax(Y)
    threshold = 0.05 * np.nanmax(Y)
    #Get indices of peak
    peak = detect_peaks(Y,edge = 'rising',mph = max_peak_height, mpd = 2, threshold = threshold )
    
    m = []
    mm = []
    for i in peak:
        m.append(Y[i]) 
        mm.append(X[i])

    mmm = np.argsort(m)
    n = []
    nn = []
    for i in mmm:
        n.append(m[i])
        nn.append(mm[i])

    n  = n[::-1]
    nn = nn[::-1]

    return n, nn

def FrequencyDomainInformation(UserInput):
    x1 = FourierTransform(UserInput)
    x2 = get_psd_values(UserInput)
    x3 = get_autocorr_values(UserInput)
    FTamp,FTfreq = GetSortedPeak(x1['Frequency'],x1['Freq. Amp.'])
    PSDamp,PSDfreq = GetSortedPeak(x2['Frequency'],x2['PSD'])
    #Cor,CorTime = GetSortedPeak(x3['X Values'],x3['Autocorr Values'])
    Cor,CorTime = [],[]
    while len(FTamp) <= 5:
        FTamp.append(['-999'])
    while len(FTfreq) <= 5:
        FTfreq.append(['-999'])
    while len(PSDamp) <= 5:
        PSDamp.append(['-999'])
    while len(PSDfreq) <= 5:
        PSDfreq.append(['-999'])
    while len(Cor) <= 5:
        Cor.append(['-999'])
    while len(CorTime) <= 5:
        CorTime.append(['-999'])
    
    x = {
        "FFT Frq @ Peak 1": FTfreq[0],
        "FFT Frq @ Peak 2": FTfreq[1],
        "FFT Frq @ Peak 3": FTfreq[2],
        "FFT Frq @ Peak 4": FTfreq[3],
        "FFT Frq @ Peak 5": FTfreq[4],
        "FFT Amp @ Peak 1": FTamp[0],
        "FFT Amp @ Peak 2": FTamp[1],
        "FFT Amp @ Peak 3": FTamp[2],
        "FFT Amp @ Peak 4": FTamp[3],
        "FFT Amp @ Peak 5": FTamp[4],
        "PSD Frq @ Peak 1": PSDfreq[0],
        "PSD Frq @ Peak 2": PSDfreq[1],
        "PSD Frq @ Peak 3": PSDfreq[2],
        "PSD Frq @ Peak 4": PSDfreq[3],
        "PSD Frq @ Peak 5": PSDfreq[4],
        "PSD Amp @ Peak 1": PSDamp[0],
        "PSD Amp @ Peak 2": PSDamp[1],
        "PSD Amp @ Peak 3": PSDamp[2],
        "PSD Amp @ Peak 4": PSDamp[3],
        "PSD Amp @ Peak 5": PSDamp[4],
        "Autocorrelate Time @ Peak 1": CorTime[0],
        "Autocorrelate Time @ Peak 2": CorTime[1],
        "Autocorrelate Time @ Peak 3": CorTime[2],
        "Autocorrelate Time @ Peak 4": CorTime[3],
        "Autocorrelate Time @ Peak 5": CorTime[4],
        "Autocorrelate @ Peak 1": Cor[0],
        "Autocorrelate @ Peak 2": Cor[1],
        "Autocorrelate @ Peak 3": Cor[2],
        "Autocorrelate @ Peak 4": Cor[3],
        "Autocorrelate @ Peak 5": Cor[4]
    }
    return x

In [5]:
def GetData(FileOfInterest):
    #Subfunction for UserInputs2WorkingForm
    data = pd.read_table(FileOfInterest,header = None)
    data.columns = ['b1x','b1y','b2x','b2y','b3x','b3y','b4x','b4y']
    return np.transpose(data.values[:,0])

def UserInputs2WorkingForm(n,N,Bd,Pd,phi,SampleFrequency,FileOfInterest,\
                           HomeDirectory,directory,TrainingDataFile):
    sig = GetData(FileOfInterest)
    NumberOfSamples = len(sig)
    dt = 1/SampleFrequency
    Tmax = dt*NumberOfSamples
    x = {
        'n': n, #Shaft rotational speed [Hz], n
        'N': N, #No. of rolling elements [-], N
        'Bd': Bd, #Diameter of a rolling element [mm], Bd
        'Pd': Pd, #Pitch diameter [mm], Pd
        'Phi': phi, #Contact angle [rad], Phi
        'Sampling Frequency': SampleFrequency,
        'Time of Sampling': Tmax,
        'Number of Samples': NumberOfSamples,
        'File of Interest': FileOfInterest,
        'HomeDirectory': HomeDirectory,
        'Working Directory': directory,
        'TrainingFileName': TrainingDataFile,
        'Signal Data of Interest': sig    
    }
    return x

In [9]:
#BEGIN HARDCODING OF GUI INFO
n = 2000 / 60
N = 16
Bd = 0.331*254
Pd = 2.815*254
phi = 15.17 * np.pi / 180
SampleFrequency = 20000
FileOfInterest = '2003.10.22.12.06.24'
HomeDirectory = os.getcwd()
os.chdir(HomeDirectory)
directory = os.listdir(HomeDirectory)
TrainingDataFile = "DELETE.csv"
#END OF HARDCODING OF GUI INFO
UserInput = UserInputs2WorkingForm(n,N,Bd,Pd,phi,SampleFrequency,FileOfInterest,HomeDirectory,directory,TrainingDataFile)
x3 = get_autocorr_values(UserInput)
Cor,CorTime = GetSortedPeak(x3['X Values'],x3['Autocorr Values'])
print(x3['X Values'])
print(x3['Autocorr Values'])
print(Cor,CorTime)

[ 0.     1.024  2.048  3.072  4.096  5.12   6.144  7.168  8.192  9.216
 10.24  11.264 12.288 13.312 14.336 15.36 ]
[3.18025808e+02 2.49938608e+02 1.81716212e+02 ... 2.40070000e-02
 6.94000000e-03 9.24000000e-04]
[] []
